# Coleta, Tratamento e Unificação das bases de dados relativas as Usinas Sucroalcooleiras

## Histórico de Atualizações

<table>
    <thead>
        <tr>
            <th>Versão</th>
            <th>Data</th>
            <th>Descrição</th>
            <th>Autor</th>
            <th>Email</th>
        </tr>
    </thead>
    <tbody>
        <tr>
        <th>1.0</th>
            <td>18/04/2023</td>
            <td>
                <p>Processo de enriquecimento das bases de Volume de Diesel Vendido CNPJ/Mês por meio de imputações de dados utilizando o método KNNInputer (sklearn)</p>
            </td>
            <td>Marcel Quintela, Sergio Urzedo Jr</td>
            <td>marcel.quintela@avanade.com, sergiourzedojr@gmail.com</td>
        </tr>
    </tbody>
</table>

# Setup Inicial

Carregando linhas com configurações iniciais já escritas em notebook base

No bloco de comando "%run" não pode ter nenhum outra linha de comando ou comentário

In [54]:
%run nb00_Setup_Usinas{'blob_relative_path_raw': 'nuvem/Usinas/'}

# Leitura dos Dados 

In [55]:
usinas   = pd.read_parquet(abfss_path_enriched+'tancagens_impt_usinas.parquet',
                            storage_options = {'linked_service' : linked_service_enriched})

arq_vol = 'abfss://general@stedlk01dtandev.dfs.core.windows.net/raw/mercado_potencial/volumetria/hist_diesel.csv'

In [56]:
prop  = pd.read_parquet('abfss://general@stedlk01dtandev.dfs.core.windows.net/enriched/mercado_potencial/volumetria/vol01_prop_consfinal_vegetativo.parquet',
                        storage_options = {'linked_service' : linked_service_enriched})
cf  = pd.read_parquet('abfss://general@stedlk01dtandev.dfs.core.windows.net/enriched/mercado_potencial/volumetria/vol01_vendas_vibra_mes.parquet',
                        storage_options = {'linked_service' : linked_service_enriched})
diesel_mes = pd.read_parquet('abfss://general@stedlk01dtandev.dfs.core.windows.net/enriched/mercado_potencial/anp/anp_diesel_mes.parquet',
                                storage_options = {'linked_service' : linked_service_enriched})

In [57]:
vol_usinas = pd.DataFrame(diesel_mes.loc['2016':]['consumidor final']*prop['USINA'][:-1])

# Ajuste do Histórico de Vendas da VIBRA

In [58]:
col_names = pd.read_csv(arq_vol, nrows=0).columns                               # coletando informações de cabeçalho
col_types = {col: str for col in col_names if col not in {}}                    # todas as colunas como str


hist = pd.read_csv(arq_vol, dtype=col_types,
                        storage_options = {'linked_service' : linked_service_raw})


hist.drop(['Unnamed: 0','index'],axis=1,inplace=True) # var não necessária
hist['VEDI_VOL'] = hist['VEDI_VOL'].astype(float)
hist['VEDI_DAT_VEN'] = pd.to_datetime(hist['VEDI_DAT_VEN'])

#organizando o Coreto!
hist = hist[['CLIE_NUM_CPF_CNPJ', 'CLIE_NOM_CLIENTE','VEDI_COD_CLI', 'VEDI_VOL', 'VEDI_DAT_VEN', 'ProdutoD', 'AECO']]

hist.rename(columns={'CLIE_NUM_CPF_CNPJ': 'CNPJ'}, errors='raise', inplace=True)


hist['CNPJ'] = hist['CNPJ'].astype(str).apply(lambda x: x.zfill(14))            # Transforma CPNJ em string
hist = hist[~hist['CNPJ'].str.startswith('000')]                                # removendo possiveis CPFs

hist = hist.merge(usinas[['CNPJ','UF']], how='left', on ='CNPJ')                # trazendo infos de Usinas Gov para filtro

hist = hist[hist['VEDI_VOL'] > 0].sort_values('VEDI_DAT_VEN')                   # Eliminando Vendas negativas ou zeradas

hist = hist[~hist['UF'].isna()].reset_index(drop=True)                          # Filtrando CNPJ que estão na base Usinas - independente da AECO

In [59]:
hist

### Agrupando vendas por mês

In [6]:
hist['ANOMES'] = hist['VEDI_DAT_VEN'].dt.strftime('%Y-%m')

hist_m = hist.groupby(['CNPJ','ANOMES'])['VEDI_VOL'].agg(Vol_Total = 'sum', Freq='count', Vol_Medio='mean').reset_index()
hist_m['data'] = pd.to_datetime(hist_m['ANOMES'], format='%Y-%m-%d')
hist_m = hist_m.set_index('data').sort_index()

### Incluindo vendas 0 nos meses que não houveram vendas

In [7]:
hist_e = pd.DataFrame()
periodo = pd.date_range(start='2016-01-01', end=datetime.today(), freq='MS')

for x in periodo.year.unique():
    per = [a for a in periodo if a.year==x]
    aux = (hist_m.loc[str(x)].groupby('CNPJ')
                 .apply(lambda x: x.reindex(per).fillna(0))  # preenche os meses sem vendas com 0
                 .rename_axis(('CNPJ','Data'))
                 .drop(['CNPJ','ANOMES'], axis=1)
                 .reset_index())
    hist_e = pd.concat([hist_e, aux], axis=0, ignore_index=True)
    
hist_e = hist_e.merge(hist[~hist['CNPJ'].duplicated()][['CNPJ','AECO']], how='left', on='CNPJ')   # resgatando a AECO

# Filtrando histórico de Clientes Vibra Contidos na Base de Usinas do GOV

In [71]:
df = usinas[['CNPJ', 'MUNICIPIO','UF', 'CAP_HIDRATADO','CAP_ANIDRO','TANCAGEM_ETANOL','TANCAGEM_DIESEL', 'SITUACAO_ANP']].merge(
       hist_e, how='left', on ='CNPJ')

df = df[~df['Data'].isna()]                                                                       # somente os clientes vibra dentro do MAPA+ANP
df['ANOMES'] = df['Data'].dt.strftime('%Y-%m')
df['ANO'] = df['Data'].dt.strftime('%Y')

# Imputação das Vendas

### Imputação Mensal

In [75]:
# Imputação ANO MES

cols = ['CNPJ','UF','REGIAO','Data', 'ANOMES', 'ANO', 'CAP_HIDRATADO','CAP_ANIDRO','TANCAGEM_ETANOL','TANCAGEM_DIESEL', 'Vol_Total','Freq','Vol_Medio']
var  = ['CAP_HIDRATADO','CAP_ANIDRO','TANCAGEM_ETANOL','TANCAGEM_DIESEL','Vol_Total','Freq','Vol_Medio']

res_m = imp_anual(df, freq='ANOMES', l=['CNPJ','Data','ANOMES', 'ANO'], cols=cols, var=var, ind=False)

res_m['ANO']= [ x[:4] for x in res_m['ANOMES']]
res_m['IMP'] = [1 if pd.isnull(x) else 0 for x in res_m['Data']]
res_m['Data'] = pd.to_datetime(res_m['ANOMES'], format='%Y-%m-%d')

In [40]:
reg = {'N':'N-NE','NE':'N-NE','S':'CO-S-SE','CO':'CO-S-SE', 'SE':'CO-S-SE'}

res_m['REGIAO2'] = [reg[x] for x in res_m['REGIAO']]
res_m['REGIAO3'] = np.where(res_m['UF']=='SP', res_m['UF'], res_m['REGIAO2'])

### Imputação Anual

In [47]:
df_ga = df.groupby(['CNPJ','ANO']).sum().reset_index()

cols = ['CNPJ','UF','REGIAO','ANO', 'CAP_HIDRATADO','CAP_ANIDRO','TANCAGEM_ETANOL','TANCAGEM_DIESEL', 'Vol_Total','Freq','Vol_Medio']
var  = ['CAP_HIDRATADO','CAP_ANIDRO','TANCAGEM_ETANOL','TANCAGEM_DIESEL','Vol_Total','Freq','Vol_Medio']

res_a = imp_anual(df_ga, 'ANO',['CNPJ','ANO'],cols, var, ind=False)

res_a['REGIAO2'] = [reg[x] for x in res_a['REGIAO']]
res_a['REGIAO3'] = np.where(res_a['UF']=='SP', res_a['UF'], res_a['REGIAO2'])

# Salvando dados

In [49]:
res_m.to_parquet(abfss_path_enriched + 'vendas_impt_mes.parquet', 
                     storage_options = {'linked_service':linked_service_enriched})
res_a.to_parquet(abfss_path_enriched + 'vendas_impt_ano.parquet', 
                     storage_options = {'linked_service':linked_service_enriched})

In [43]:
def agrup_graf(df, geo='UF'):
    aux = df.set_index('Data')
    aux = aux.groupby([geo])['Vol_Total'].resample('MS').sum()
    aux = aux.unstack(level=0)
    #aux.columns = aux.columns.droplevel(0)
    aux = aux.rename_axis(None, axis=1)
    return(aux)

In [45]:
multi_plot(agrup_graf(res_m, geo='REGIAO2'))

# TAMANHO DE MERCADO

VISÃO ANP

In [503]:
cf = pd.DataFrame(diesel_mes.loc['2016':]['consumidor final'])
cf["ANO"] = cf.index.strftime('%Y')
cf = cf.groupby(['ANO']).sum().reset_index()

# TAMANHO DE MERCADO DA USINAS = VOLUMES CONF VIBRA DA ANP * %USINAS VIBRA
vol_usinas = pd.DataFrame(diesel_mes.loc['2016':]['consumidor final']*prop['USINA'][:-1])
vol_usinas.columns = ['usina']

# Volume de diesel vendido para usina ANO GLOBAL
vol_usinas['ANO'] = vol_usinas.index.strftime('%Y')
vol_usinas = vol_usinas.groupby(['ANO']).sum().reset_index()

vol_usinas['cf'] = cf['consumidor final'].to_list()

vol_usinas['perc'] = vol_usinas['usina']/vol_usinas['cf']
vol_usinas

In [514]:
# ERROS RELATIVOS AO TAMANHO DO MERCADO 
((a['Vol_Total'] - vol_usinas['usina'])/vol_usinas['usina']).plot()

In [473]:
vol_usinas.plot()

In [447]:
# TAMANHO DE MERCADO ESTIMADO POR MEIO DE CLIENTES E NÃO CLIENTES VIBRA
# PROCESSO DE IMPUTAÇÕES DE VOLUMES VENDIDOS PARA' NÃO CLIENTES
b1.plot()

In [450]:
# VENDAS VIBRA REAL PARA USINAS
c1.plot()

In [428]:
a1 = a.merge(b, on='ANO')
a1['Diff'] = a1['Vol_Total_x'].sub(a1['Vol_Total_y'], axis=0)
a1

### COLETANDO VOLUME DECOMPOSTO

In [448]:
vol = pd.read_parquet('abfss://general@stedlk02dtandev.dfs.core.windows.net/enriched/mercado_potencial/volumetria/vol02_MKTS_ConsFinal_Decomposto.parquet',
                        storage_options = {'linked_service' : linked_service_enriched})

vol['ANOMES'] = vol.index.strftime('%Y-%m')
vol['ANO'] = vol.index.strftime('%Y')  

c = vol.groupby(['ANO'])['USINA'].agg(Vol_Total = 'sum').reset_index()
c1 = vol.groupby(['ANOMES'])['USINA'].agg(Vol_Total = 'sum').reset_index()
c

In [432]:
c['c-a'] = c['Vol_Total'] - a['Vol_Total']
c['c-b'] = c['Vol_Total'] - b['Vol_Total']
c